1. Thu thập các đường dẫn thuộc mỗi chuyên mục từ trang web https://vnexpress.net/

- Viết hàm lấy tất cả các đường dẫn của bài báo trong trang web

In [1]:
# Gọi thư viện
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import re

# Đường dẫn đến trang web gốc
URL = 'https://vnexpress.net/'

def _get_links(url):
    links = [] 
    html = urlopen(url).read()  
    soup = BeautifulSoup(html, 'html.parser') 

    # Lấy tất cả đường dẫn trong trang web hiện tại
    urls = set(re.findall(r'(?:http|https|ftp):\/\/(?:[\w_-]+(?:\.[\w_-]+)+)(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', str(soup)))

    for link in urls:
        # Chỉ lấy đường dẫn là trang web nhiều bài báo
        if link != URL and link.startswith(URL) and not link.startswith(URL + 'tac-gia') and link.endswith('.html'):
            links.append(link)

    return links

- Kết quả trả về của hàm với đường dẫn gốc của VNexpress

In [2]:
_get_links(URL)

['https://vnexpress.net/hoc-de-song-hay-hoc-de-thi-4808158.html',
 'https://vnexpress.net/hang-tram-nghin-smartphone-xach-tay-khong-dung-duoc-5g-o-viet-nam-4808035.html',
 'https://vnexpress.net/hang-nghin-ten-mien-vn-bi-dung-de-lua-dao-4808000.html',
 'https://vnexpress.net/chay-cua-hang-trong-cho-ba-chieu-4808160.html',
 'https://vnexpress.net/israel-tuyen-bo-ha-chi-huy-hamas-lam-viec-trong-co-quan-lhq-4808184.html',
 'https://vnexpress.net/that-vong-voi-chung-cu-duoc-quang-cao-cao-cap-4807729.html',
 'https://vnexpress.net/thu-tuong-pham-minh-chinh-hoi-dam-voi-tong-thong-putin-4808261.html',
 'https://vnexpress.net/hamas-neu-dieu-kien-ngung-giao-tranh-o-dai-gaza-4808182.html',
 'https://vnexpress.net/lang-viet-co-o-ninh-binh-hoang-tan-4807636.html',
 'https://vnexpress.net/bao-tra-mi-manh-len-cap-10-4808210.html',
 'https://vnexpress.net/lung-tung-tim-huong-xu-ly-vu-mat-nha-dat-sau-nhieu-nam-xa-que-4808061.html',
 'https://vnexpress.net/belarus-muon-hop-tac-san-xuat-oto-cung-viet-na

- Lấy đường dẫn của các bài toán thuộc từng chuyên mục

In [3]:
# Danh sách các chuyên mục
categories = [
    'thoi-su', 'kinh-doanh', 'khoa-hoc', 'giai-tri',
    'the-thao', 'phap-luat', 'giao-duc', 'suc-khoe',
    'doi-song', 'du-lich', 'so-hoa', 'oto-xe-may'
]

# Danh sách các đường dẫn đến các bài báo của mỗi chuyên mục
article_url = {}

for category in categories:
    article_url[category] = []

    # Lấy 10 trang trong mỗi chuyên mục
    for i in range(1, 11): 
        url = f'{URL}{category}-p{i}'
        article_url[category] += _get_links(url)

    # Loại bỏ đường dẫn trùng nhau
    article_url[category] = list(set(article_url[category]))

    # Hiển thị số đường dẫn thu thập được trong chuyên mục
    print(category, len(article_url[category]))

# Lấy riêng các đường dẫn trong chuyên mục theo góc nhìn từng mục con
goc_nhin = [
    'binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-su-khoe',
    'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong',
    'van-hoa-loi-song', 'covid-19'
]

article_url['goc-nhin'] = []

for sub_cate in goc_nhin:
    url = f'{URL}/goc-nhin/{sub_cate}'
    article_url['goc-nhin'] += _get_links(url)

# Loại bỏ đường dẫn trùng nhau trong chuyên mục 'goc-nhin'
article_url['goc-nhin'] = list(set(article_url['goc-nhin']))
print('goc-nhin', len(article_url['goc-nhin']))


thoi-su 316
kinh-doanh 315
khoa-hoc 305
giai-tri 300
the-thao 305
phap-luat 314
giao-duc 322
suc-khoe 301
doi-song 307
du-lich 326
so-hoa 303
oto-xe-may 298
goc-nhin 24


2. Lấy nội dung của bài báo ứng với từng đường dẫn cụ thể và lưu vào một file trong folder chuyên mục tương ứng.

- Viết hàm lấy nội dung của bài báo ứng với đường dẫn cụ thể

In [4]:
def _get_content(url):
    # Nội dung bài báo
    content = ''
    
    # Mở URL và đọc nội dung trang
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    # Lấy phần chứa nội dung bài báo
    div_content = soup.select('.page-detail .container')
    if len(div_content) > 0:
        div_content = div_content[0]

        # Lấy phần mô tả
        description = div_content.find_all('p', {'class': 'description'})
        if len(description) > 0:
            description = description[-1]
            text_description = description.get_text()
            location = description.find('span', {'class': 'location-stamp'})
            if location is not None:
                content = text_description[len(location.get_text()):]
            else:
                content = text_description   

        # Lấy phần chứa nội dung chi tiết bài báo
        detail = div_content.find('article', {'class': 'fck_detail'})
        if detail is not None:
            p_normal = detail.find_all('p', {'class': 'Normal'})
            if len(p_normal) > 0:
                for p in p_normal:
                    p_text = p.get_text()
                    # Kiểm tra đoạn văn bản không rỗng, không bắt đầu bằng dấu '>>' và kết thúc bằng các ký tự hợp lệ
                    if p_text != '' and not p_text.startswith('>>') and p_text[-1] in ['.', '!', '?']:
                        content += ' ' + p.get_text()

    # Xóa ký tự ngắt dòng và trả về
    return re.sub(r'\n', '', content)

+ Kết quả trả về của hàm với đường dẫn dầu tiên trong chuyên mục số hóa

In [5]:
_get_content(article_url['thoi-su'][0])

'Thành phố dự tính phát triển mảng xanh dọc sông Sài Gòn với 42 công viên, kết hợp đầu tư hạ tầng đa chức năng giúp cải tạo cảnh quan, phát triển kinh tế, dịch vụ. Đây là một trong những nội dung nêu trong báo cáo tổng kết của UBND TP HCM về đề án Phát triển kè sông và kinh tế dịch vụ ven sông Sài Gòn, giai đoạn 2020-2045. Phạm vi nghiên cứu của đề án là dọc theo hành lang sông Sài Gòn đoạn chảy qua địa bàn thành phố, gồm TP Thủ Đức, quận 1, 4, 7, 12, Bình Thạnh và hai huyện Củ Chi, Hóc Môn. Định hướng phát triển chuỗi công viên thuộc nhóm xây dựng hạ tầng xanh đa chức năng, kết nối giao thông và triển khai các dự án thành phần ven sông. Kế hoạch này được đưa ra sau khi các chuyên gia và tư vấn nghiên cứu, đề xuất tích hợp vào điều chỉnh quy hoạch chung TP HCM. Các công viên khi hình thành được kỳ vọng phát huy lợi thế, tiềm năng sông Sài Gòn trong phát triển kinh tế dịch vụ, chất lượng cảnh quan... Việc triển khai các dự án sẽ phân kỳ, phân đoạn, phân vùng, gắn với giao thông và hạ tầ

- Lưu nội dung các bài báo thành các file nằm trong folder chuyên mục tương ứng

In [6]:
import os

# Danh sách tất cả các câu đã làm sạch của tất cả các bài viết
sentence = []
#Chữ cái in hoa của tiếng việt
uppercase = "ABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ"

for category in article_url.keys():
    # Tạo folder chuyên mục để chứa các file bài viết
    current_path = 'data/categories/' + category + '/'
    os.mkdir(current_path)

# Lưu các bài viết thành từng file
    count = 0
    for i in range(len(article_url[category])):
        content = _get_content(article_url[category][i])
        if content != '':
            count += 1
            f = open('{}/{}_{}.txt'.format(current_path, category, str(count)), 'w', encoding='utf-8') 
            # Chèn khoảng cách vào sau các dấu câu (., ?, !) nếu không có khoảng trắng hoặc dấu ngoặc kép ngay sau đó
            lines = re.sub(r'(?<=[.?!])(?=\s*[\'"`"’]*[{}])'.format(uppercase), ' ', content)

            # Tách bài viết thành các câu (giả sử câu kết thúc bằng ., !, ? và câu mới bắt đầu bằng chữ cái in hoa hoặc chữ số)
            lines = re.split(r'(?<=[.?!])(?=\s*[{}0-9])'.format(uppercase), content)

            # Ghi từng câu trong vào viết vào file
            for line in lines:
                f.write((line + '\n'))
                sentence.append(line)
            f.close()
        # Sổ bài viết trong chuyên mục    
        print(category, count)

        sentence = list(set(sentence))
        print("Tổng số câu được tách: ", len(sentence))

thoi-su 1
Tổng số câu được tách:  24
thoi-su 2
Tổng số câu được tách:  65
thoi-su 3
Tổng số câu được tách:  84
thoi-su 4
Tổng số câu được tách:  104
thoi-su 5
Tổng số câu được tách:  114
thoi-su 6
Tổng số câu được tách:  126
thoi-su 7
Tổng số câu được tách:  150
thoi-su 8
Tổng số câu được tách:  163
thoi-su 9
Tổng số câu được tách:  181
thoi-su 10
Tổng số câu được tách:  187
thoi-su 11
Tổng số câu được tách:  196
thoi-su 12
Tổng số câu được tách:  209
thoi-su 13
Tổng số câu được tách:  224
thoi-su 14
Tổng số câu được tách:  236
thoi-su 15
Tổng số câu được tách:  251
thoi-su 16
Tổng số câu được tách:  260
thoi-su 17
Tổng số câu được tách:  276
thoi-su 18
Tổng số câu được tách:  286
thoi-su 19
Tổng số câu được tách:  313
thoi-su 20
Tổng số câu được tách:  327
thoi-su 21
Tổng số câu được tách:  353
thoi-su 22
Tổng số câu được tách:  360
thoi-su 23
Tổng số câu được tách:  379
thoi-su 24
Tổng số câu được tách:  394
thoi-su 25
Tổng số câu được tách:  422
thoi-su 26
Tổng số câu được tách:  44

+ 10 câu dầu tiên tách được

In [7]:
sentence[:10]

[' Trước đó, Hoàng Việt Travel và một số công ty khác trên thị trường cũng thực hiện tour khảo sát dịp hè ở Moskva, St.',
 ' Để bảo vệ gan, bác sĩ Hà khuyến cáo mỗi người cần duy trì thói quen ăn uống lành mạnh, tập thể dục đều đặn và khám sức khỏe định kỳ.',
 ' Nghi can sau đó lĩnh thêm hai án tù liên quan đánh bạc.',
 ' Chuyến đi một ngày "chén sạch Hải Phòng" với các món ăn tiêu biểu như bánh đa cua, bánh mì cay, sủi dìn, giá bể, cà phê cốt dừa, bánh đúc tàu, trà cúc, các loại ốc sẽ là gợi ý phù hợp cho những du khách có "tâm hồn ăn uống".',
 ' Michael O\'Regan, giảng viên Đại học Glasgow Caledonian ở Scotland, cho rằng thuật ngữ trên không phản ánh được thực tế bất lực trong việc quản lý đám đông hiện nay. "Du lịch đã quay trở lại nhanh hơn chúng tôi mong đợi", Micheal nói.',
 ' Ba bàn và hai đường kiến tạo trong một trận đấu cũng là điều Ronaldo chưa từng làm được trong sự nghiệp, trong khi Messi đã trải qua bốn trận như vậy, năm 2010, 2011, 2018 và 2024.',
 ' Ngày 21/10, Ngân hàn

<strong>3. Lưu tất cả các câu vào file ngữ liệu </strong>

In [8]:
# File ngữ liệu
file_corpus = open('data/corpus.txt', 'w', encoding='utf-8')

count = 0
for sent in sentence:
    # Xóa ký tự thừa
    sent = re.sub(r'\u200b', '', sent)

    file_corpus.write(sent.strip() + '\n')
    count +=1
file_corpus.close()

print('Số lượng câu trong file ngữ liệu: ', count)

Số lượng câu trong file ngữ liệu:  82105
